# SQL

## Задачи проекта
В рамках проекта необходимо проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательских обзорах книг, сформулировать преимущества существующего сервиса для чтения книг.

## Задания
Посчитайте, сколько книг вышло после 1 января 2000 года;\
Для каждой книги посчитайте количество обзоров и среднюю оценку;\
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц;\
Определите автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;\
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе данных с помощью SQLalchemy

In [6]:
#загрузим библиотеки
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=False)

## Описание данных:

### Таблица books

Содержит данные о книгах:

book_id — идентификатор книги;\
author_id — идентификатор автора;\
title — название книги;\
num_pages — количество страниц;\
publication_date — дата публикации книги;\
publisher_id — идентификатор издателя.

In [8]:
# Проверка загрузки таблицы с указанием книг
query = '''
            SELECT * FROM books
        '''

books = pd.io.sql.read_sql(query, con = engine)

display(books.head())
display(books.info())

c:\Users\kirio\anaconda3\envs\da_practicum_env2\lib\site-packages\pandas\io\sql.py:66: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(sqlalchemy.__version__) >= LooseVersion("1.4.0")


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

### Таблица authors

Содержит данные об авторах:

author_id — идентификатор автора;\
author — имя автора.

In [9]:
# Проверка загрузки таблицы с указанием авторов
query = '''
            SELECT * FROM authors 
        '''

authors = pd.io.sql.read_sql(query, con = engine)

display(authors.head())
display(authors.info())

c:\Users\kirio\anaconda3\envs\da_practicum_env2\lib\site-packages\pandas\io\sql.py:66: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(sqlalchemy.__version__) >= LooseVersion("1.4.0")


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

### Таблица publishers

Содержит данные об издательствах:

publisher_id — идентификатор издательства;\
publisher — название издательства;

In [10]:
# Проверка загрузки таблицы с информацией об издательствах

query = '''
            SELECT * FROM  publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)

display(publishers.head())
display(publishers.info())

c:\Users\kirio\anaconda3\envs\da_practicum_env2\lib\site-packages\pandas\io\sql.py:66: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(sqlalchemy.__version__) >= LooseVersion("1.4.0")


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

### Таблица ratings

Содержит данные о пользовательских оценках книг:

rating_id — идентификатор оценки;\
book_id — идентификатор книги;\
username — имя пользователя, оставившего оценку;\
rating — оценка книги.

In [11]:
# Проверка загрузки таблицы с информацией о рейтингах

query = '''
            SELECT * FROM  ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)

display(ratings.head())
display(ratings.info())

c:\Users\kirio\anaconda3\envs\da_practicum_env2\lib\site-packages\pandas\io\sql.py:66: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(sqlalchemy.__version__) >= LooseVersion("1.4.0")


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

### Таблица reviews

Содержит данные о пользовательских обзорах на книги:

review_id — идентификатор обзора;\
book_id — идентификатор книги;\
username — имя пользователя, написавшего обзор;\
text — текст обзора.

In [12]:
# Проверка таблицы c информацией об обзорах 

query = '''
            SELECT * FROM  reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)

display(reviews.head())
display(reviews.info())

c:\Users\kirio\anaconda3\envs\da_practicum_env2\lib\site-packages\pandas\io\sql.py:66: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(sqlalchemy.__version__) >= LooseVersion("1.4.0")


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None